This notebook computes phrases in data and replaces words with phrases. Also trains a new Word2Vec embeddings upon those phrased data.

Use either "Prepare Data 20news" part or "Prepare data fold0" part.

In [ ]:
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser
import gensim
import newsgroups
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import numpy as np
import utility
import pickle
import datetime


# Prepare Data 20news


In [2]:
trainData, trainTarget, testData, testTarget, class_dict, reverse_class_dict = newsgroups.process_newsgroups_numpy(True,False,True,False )

In [3]:
should_fetch = True
should_save = True
should_read = True

In [4]:
if should_fetch:
    # Fetch data
    raw_data_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
    target_names = raw_data_train.target_names
    raw_data_train = pd.DataFrame([raw_data_train.data, raw_data_train.target.tolist()]).T
    raw_data_train.columns = ['text', 'target']

    raw_data_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
    raw_data_test = pd.DataFrame([raw_data_test.data, raw_data_test.target.tolist()]).T
    raw_data_test.columns = ['text', 'target']

if should_save:
    raw_data_train.to_csv("lol_train.csv")
    raw_data_test.to_csv("lol_test.csv")

if should_read:
    # Read data
    trainData = pd.read_csv("lol_train.csv")
    testData = pd.read_csv("lol_test.csv")
    trainData = trainData.values
    testData = testData.values

# Remove extra dimension
trainData[:,0] = trainData[:,2]
trainData = trainData[:,1:3]
testData[:,0] = testData[:,2]
testData = testData[:,1:3]

trainData_raw = trainData
testData_raw = testData

# Clean Data
trainData[:,0] = [utility.clean_str(a).split(" ") for a in trainData[:,0]]
testData[:,0] = [utility.clean_str(a).split(" ") for a in testData[:,0]]

# Split data and labels(Target)
trainTarget = trainData[:,1]
trainData = trainData[:,0]
testTarget = testData[:,1]
testData = testData[:,0]



In [5]:
trainData_raw = [utility.clean_str(a) for a in trainData_raw]
testData_raw = [utility.clean_str(a) for a in testData_raw]

# Prepare data fold0

In [7]:
configs = {
    "vectorSize":300,
    "trainNewModel":True,
    "dataColumn":"question",
    "maxLength":400,
    "batchSize":8,
    "embeddingType":None,
    "ELMo":True,
    "PreEmbed":True,
    "restore":True
}

In [8]:
dataColumn = "question"
fold0trainRaw = pandas.read_csv("Data/fold_0_train_tr.csv", encoding="utf-8")
fold0testRaw = pandas.read_csv("Data/fold_0_test_tr.csv")
fold0trainData = np.hstack((fold0trainRaw[dataColumn].values.reshape(-1,1),fold0trainRaw["category2"].values.reshape(-1,1)))
fold0testData = np.hstack((fold0testRaw[dataColumn].values.reshape(-1,1),fold0testRaw["category2"].values.reshape(-1,1)))
fold0ClassDict = DataLoader.DataHandler.getUniqueClassMapDict(fold0testData[:,1])

    
trainData = fold0trainData
trainData, trainTarget, _ = DataLoader.DataHandler.masterPreprocessor(trainData,shuffle=True,classDict=fold0ClassDict,maxLength=configs["maxLength"])
testData, testTarget, _ = DataLoader.DataHandler.masterPreprocessor(fold0testData,shuffle=True,classDict=fold0ClassDict,maxLength=configs["maxLength"])


Outputs converted to numerical forms
Input text claned
Input text split into tokens and all inputs padded to maximum length
Outputs converted to numerical forms
Input text claned
Input text split into tokens and all inputs padded to maximum length


In [10]:
all_data = np.array(list(trainData)+list(testData))

# Get Phrases (BiGram and TriGram)

In [ ]:
import pandas
import DataLoader

In [11]:
testData = all_data
# Convert test data to bigram-trigram
test_bigram_model = Phraser(Phrases(testData, min_count=10, threshold=100))
test_bigram = [test_bigram_model[line] for line in testData]
test_trigram_model = Phraser(Phrases(test_bigram, min_count=10, threshold=100))
test_trigram = [test_trigram_model[line] for line in test_bigram]

In [14]:
# Convert train data to bigram-trigram
train_bigram_model = Phraser(Phrases(trainData, min_count=10, threshold=100))
# change data with phrases
train_bigram = [train_bigram_model[line] for line in trainData]


train_trigram_model = Phraser(Phrases(train_bigram, min_count=10, threshold=100))
# change data with phrases
train_trigram = [train_trigram_model[line] for line in train_bigram]

In [57]:
train_trigram = np.array(train_trigram)
test_trigram = np.array(test_trigram)

In [58]:
train_data_trigram = np.vstack((train_trigram, trainTarget)).T
test_data_trigram = np.vstack((test_trigram, testTarget)).T

In [65]:
# if you do not want to save just change it to false
should_save = True
if should_save:
    pd.DataFrame(train_trigram).to_csv("data//icliniq//train_trigram.csv")
    pd.DataFrame(test_trigram).to_csv("data//icliniq//test_trigram.csv")
pickle.dump(trainTarget, open( "data//icliniq//train_trigram_target.pkl", "wb" ))
pickle.dump(testTarget, open( "data//icliniq//test_trigram_target.pkl", "wb" ))

In [13]:
phrases = []
for i in range(len(testData)):
    for phrase in test_trigram_model[test_bigram[i]]:
        if phrase is not "[None]" and "_" in phrase and phrase not in phrases:
            phrases.append(phrase)
print(len(phrases))

200


In [ ]:
with open("phrases_icliniq.txt", "w") as f:
    for item in phrases:
        f.write("%s\n" % item)

In [25]:
# Change encoding (there may be error otherwise)
for i in range(len(test_trigram)):
    test_trigram[i] = [a.encode("utf-8") for a in test_trigram[i]]

# Train Word2Vec with phrased data

In [17]:
word_vectors = w2v_model.wv

In [24]:
all_data = [list(a) for a in all_data] 
data = all_data
w2v_model = gensim.models.Word2Vec(data,size=300, window=10, min_count=2,workers=10)
w2v_model.train(data, total_examples=len(data), epochs=10)

(7172539, 28712000)

In [25]:
w2v_model.wv.save("Embeddings//icliniq_default//icliniq_default.w2v")